In [3]:
import pandas as pd
import pyodbc
import sqlite3

C:\Users\daisy\AppData\Local\Temp\ipykernel_11828\2698988305.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
DB = {"servername": "DESKTOP-OF0CT86\SQLEXPRESS",
      "database": "greatoutdoors"}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()


# Load utilities


# Load tables

sales_db = sqlite3.connect("Databases/GreatOutdoors/go_sales.sqlite")
product = pd.read_sql_query("SELECT * FROM product;", sales_db)
country = pd.read_sql_query("SELECT * FROM country;", sales_db)
product_line = pd.read_sql_query("SELECT * FROM product_line;", sales_db)

sales_branch = pd.read_sql_query("SELECT * FROM sales_branch;", sales_db)
retailer_site = pd.read_sql_query("SELECT * FROM retailer_site;", sales_db)
product_type = pd.read_sql_query("SELECT * FROM product_type;", sales_db)
returns = pd.read_sql_query("SELECT * FROM returned_item;", sales_db)
order_details = pd.read_sql_query("SELECT * FROM order_details;", sales_db)
sales_staff = pd.read_sql_query("SELECT * FROM sales_staff;", sales_db)
order_header = pd.read_sql_query("SELECT * FROM order_header;", sales_db)
returned_item  = pd.read_sql_query("SELECT * FROM returned_item;", sales_db)
return_reason  = pd.read_sql_query("SELECT * FROM return_reason;", sales_db)
target = pd.read_sql_query("SELECT * FROM SALES_TARGETData;", sales_db)
order_method = pd.read_sql_query("SELECT * FROM order_method;", sales_db)



for row in export_cursor.tables():
    print(row.table_name)


In [ ]:
# Product
transformed_product = []

transformed_product = product.merge(product_type, 'inner', 'PRODUCT_TYPE_CODE').merge(product_line, 'inner', 'PRODUCT_LINE_CODE').rename(columns={
    "INTRODUCTION_DATE": "INTRODUCTION_DATE_date",
    "PRODUCT_TYPE_CODE": "PRODUCT_TYPE_CODE",
    "PRODUCTION_COST": "PRODUCTION_COST_number",
    "MARGIN": "MARGIN_NUMBER",
    "PRODUCT_IMAGE": "PRODUCT_IMAGE_image",
    "LANGUAGE": "LANGUAGE_language",
    "PRODUCT_TYPE_EN": "PRODUCT_TYPE_name",
    "DESCRIPTION": "DESCRIPTION_description",
}).drop(columns="TRIAL888_x", axis=1).drop(columns="TRIAL888_y", axis=1).drop(columns="TRIAL888", axis=1)


for index, row in transformed_product.iterrows():
    try:
        query = f"INSERT INTO Product VALUES ({row['PRODUCT_NUMBER']},  '{row['PRODUCT_NAME']}', '{ convert_date_format(row['INTRODUCTION_DATE_date']) }', {row['PRODUCT_TYPE_CODE']}, '{row['PRODUCT_TYPE_name']}', {row['PRODUCT_LINE_CODE']}, '{row['PRODUCT_LINE_EN']}', {row['PRODUCTION_COST_number']}, {row['MARGIN_NUMBER']},'{row['PRODUCT_IMAGE_image']}', '{row['LANGUAGE_language']}', '{escape_single_quotes(row['DESCRIPTION_description'])}');"
        export_cursor.execute(query)
        print(query)
    except pyodbc.Error as e:
        print(query)
        # print(e)

    export_cursor.commit()


In [ ]:
# Order


merged_df = pd.merge(order_header, order_details, on='ORDER_NUMBER', how='inner')
order = pd.merge(merged_df, returned_item, on='ORDER_DETAIL_CODE', how = 'inner')

for index, row in order.iterrows():
    try:
        query = f"INSERT INTO 'orders' VALUES ({row['ORDER_NUMBER']},  '{escape_single_quotes(row['RETAILER_NAME'])}', '{row['ORDER_DATE']}', 1, 1, 2024, {row['ORDER_DETAIL_CODE']}, {row['PRODUCT_NUMBER']}, {row['QUANTITY']}, {row['UNIT_COST']}, {row['UNIT_PRICE']}, {row['UNIT_SALE_PRICE']}, {row['RETURN_CODE']}, '{convert_date_format(strip_time_from_string(row['RETURN_DATE']))}', {row['RETURN_REASON_CODE']}, {row['RETURN_QUANTITY']}, {row['ORDER_METHOD_CODE']}, {row['SALES_STAFF_CODE']}, {row['SALES_BRANCH_CODE']}, {row['RETAILER_SITE_CODE']});"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)
        
order

In [ ]:
# Sales Forecast & Inventory

sales_data_forecast = pd.read_csv("Databases/GreatOutdoors/GO_SALES_PRODUCT_FORECASTData.csv")
sales_inventory_levels = pd.read_csv("Databases/GreatOutdoors/GO_SALES_INVENTORY_LEVELSData.csv")

sales_data_merged = pd.merge(sales_data_forecast, sales_inventory_levels, 
                     left_on=['PRODUCT_NUMBER','YEAR','MONTH'],
                     right_on=['PRODUCT_NUMBER','INVENTORY_YEAR','INVENTORY_MONTH'],
                     how='inner')



sales_data_merged = sales_data_merged.rename(columns={'YEAR': 'FORECAST_YEAR', 'MONTH': 'FORECAST_MONTH'})



for index, row in sales_data_merged.iterrows():
    try:
        query = f"INSERT INTO 'SALESDATA' VALUES ({row['PRODUCT_NUMBER']}, {row['FORECAST_MONTH']}, {row['FORECAST_YEAR']}, {row['INVENTORY_MONTH']}, {row['INVENTORY_YEAR']}, {row['INVENTORY_COUNT']});"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)


# len(sales_data_merged)




In [6]:
# Order Method

for index, row in order_method.iterrows():
    try:
        query = f"INSERT INTO order_method VALUES ({row['ORDER_METHOD_CODE']}, '{row['ORDER_METHOD_EN']}');"
        print(query)
    except pyodbc.Error as e:
        print(query)
        print(e)

INSERT INTO order_method VALUES (1, 'Fax');
INSERT INTO order_method VALUES (2, 'Telephone');
INSERT INTO order_method VALUES (3, 'Mail');
INSERT INTO order_method VALUES (4, 'E-mail');
INSERT INTO order_method VALUES (5, 'Web');
INSERT INTO order_method VALUES (7, 'Sales visit');
INSERT INTO order_method VALUES (8, 'Special');
